This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [4]:
# If haven't downloaded it yet, please get the data file with wget

!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2021-11-15 13:15:15--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com... 162.125.69.18
Connecting to www.dropbox.com|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2021-11-15 13:15:15--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1bfb60a998fffc5bc0979ea0cd.dl.dropboxusercontent.com/cd/0/inline/BaAHFO-USVxqAMVOaTJe7ss3G_onD51YJbtVKiWRCFA_Nz_RrYmDqKs3mFF06tJHxqZbGIDpmoONErZci0Dr4rfzwL63eJMiEx9iqa2WoDe9tA0_Lz9s5jLh13AQ7vS5ih9QFXYHKvrbdwpj2xWdBVmV/file# [following]
--2021-11-15 13:15:15--  https://uc1bfb60a998fffc5bc0979ea0cd.dl.dropboxusercontent.com/cd/0/inline/BaAHFO-USVxqAMVOaTJe7ss3G_onD51YJbtVKiWRCFA_Nz_RrYmDqKs3mFF06tJHxqZbGIDpmoONErZci0Dr4rfzwL63eJMiEx9iqa2WoDe9tA0_Lz9s5jLh13AQ7vS5ih9QFXYHKvrbdwp

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/data_000637.txt', sep=",", names=['HEAD', 'FPGA', 'TDC_CHANNEL', 'ORBIT_CNT', 'BX_COUNTER', 'TDC_MEAS'])
print(df)


         HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
1           1     0          123  3869200167        2374        26
2           1     0          124  3869200167        2374        27
3           1     0           63  3869200167        2553        28
4           1     0           64  3869200167        2558        19
...       ...   ...          ...         ...         ...       ...
1310716     1     0           62  3869211171         762        14
1310717     1     1            4  3869211171         763        11
1310718     1     0           64  3869211171         764         0
1310719     1     0          139  3869211171         769         0
1310720     1     0           61  3869211171         762        18

[1310721 rows x 6 columns]


/Users/dbiyikli/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [23]:
#Maximum of the BX_COUNTER values should give the closest value
#Since the values are string, astype() has been used to convert. Of course, max() gives error with the string values.
#print(df[1::]['BX_COUNTER'].astype(int).max())

BX_COUNTER_max = df[1::]['BX_COUNTER'].astype(int).max() +1 # Add 1 and choose BX_COUNTER_max as maximum. 
print(BX_COUNTER_max)

3564


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [40]:
import datetime
#print(df.loc[1::,['ORBIT_CNT','BX_COUNTER','TDC_MEAS']])

#print(int(df.at[1,'ORBIT_CNT']))
#print(int(df.at[len(df)-1,'ORBIT_CNT']))
Time_in_Ns = ((int(df.at[len(df)-1,'ORBIT_CNT'])-int(df.at[1,'ORBIT_CNT']))*BX_COUNTER_max)*25 

# for BX_COUNTER column the data starts with 2374 
# And ends with 762, which means they are complements of each other,
# Therefore, we can consider only the ORBIT_CNT column.
# Also, TDC_MEAS is very small in seconds.
#print(type(Time_in_Ns))
print(pd.to_timedelta(str(Time_in_Ns)))

0 days 00:00:00.980456400


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [17]:
ns_1 = [(int(df.at[i,'ORBIT_CNT'])*BX_COUNTER_max)*25+(int(df.at[i,'BX_COUNTER']))*25+int(df.at[i,'TDC_MEAS'])*25/30 for i in range(1,len(df))] 
ns=["NS"]+ns_1
#print(len(ns))
#print(ns)
#print(int(df.at[1,'ORBIT_CNT']))
df['ns'] = pd.Series(ns, index=df.index)
print(df)

         HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS   
1           1     0          123  3869200167        2374        26   
2           1     0          124  3869200167        2374        27   
3           1     0           63  3869200167        2553        28   
4           1     0           64  3869200167        2558        19   
...       ...   ...          ...         ...         ...       ...   
1310716     1     0           62  3869211171         762        14   
1310717     1     1            4  3869211171         763        11   
1310718     1     0           64  3869211171         764         0   
1310719     1     0          139  3869211171         769         0   
1310720     1     0           61  3869211171         762        18   

                           ns  
0                          NS  
1        344649004934896.6875  
2           344649004934897.5  
3        344649004939373.3125  

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [20]:
print(df[1::].groupby('TDC_CHANNEL').sum().sort_index(axis=1,ascending=False).head(3))


                       ns TDC_MEAS        ORBIT_CNT   HEAD   FPGA BX_COUNTER
TDC_CHANNEL                                                                 
1            9.207659e+18   388338  103369728951189  26716  25621   47759492
2            1.064140e+19   449842  119465629426900  30876  29417   55010461
3            7.283135e+18   306448   81764077846457  21132  19791   37584056


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [78]:
TDC_CHANNEL_max = df[1::]['TDC_CHANNEL'].astype(int).max()
#print(TDC_CHANNEL_max)
#print(int(df.at[1,'FPGA']))
TDC_CHANNEL_List = np.arange(1,TDC_CHANNEL_max+1)
Filtered_TDC_CHANNEL_List = []
[Filtered_TDC_CHANNEL_List.append(int(df.at[i,'TDC_CHANNEL'])) for i in range(1,len(df)-1)] 
#print(np.unique(Filtered_TDC_CHANNEL_List)) # Non-Empty Orbits
print("\n The number of non empty orbits: \n", len(np.unique(Filtered_TDC_CHANNEL_List)))
#print(list(set(TDC_CHANNEL_List)-set(np.unique(Filtered_TDC_CHANNEL_List)))) #Empty Orbits
TDC139_List = []
[TDC139_List.append(int(df.at[j,'ORBIT_CNT'])) for j in range(1,len(df)-1) if int(df.at[j,'TDC_CHANNEL'])==139]
print("\n The number of unique orbits with at least one measurement from TDC = 139 is: \n", len(np.unique(TDC139_List)))
      


 The number of non empty orbits: 
 133

 The number of unique orbits with at least one measurement from TDC = 139 is: 
 10976


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

In [ ]:
#[TDC139_List.append(int(df.at[j,'ORBIT_CNT'])) for j in range(1,len(df)-1) if int(df.at[j,'TDC_CHANNEL'])==139]
#Bunu her bir TDC Channel için yapıp çizdirmek gerekiyor.
# İki plot olacak FPGA = 0 ve FPGA = 1
